In [23]:
from scipy.optimize import differential_evolution
import numpy as np
import json
import matplotlib.pyplot as plt
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import tellurium as te
import json
plt.rcParams["font.family"] = "serif"
plt.style.use('seaborn-deep')
plt.rcParams["font.serif"] = ["Times New Roman"] + plt.rcParams["font.serif"]
%config Completer.use_jedi = False
showOriginalModelString = True
free_params = {
    'IL10_det':[10,100], # detrimental threshold for >48 exposure
    'maturity_t':[0,1], # early maturity threshold
    'diff_L':[0.05,0.45], # center of low membership function
    'diff_H':[0.55,0.95], # center of high membership function
    'a_early_diff':[1,100], # scale factor
    'a_late_diff':[1,100], # scale factor
    'diff_time':[15,45], # days required for full differentiation
    'ALP_maturity': [1,1000], #TODO: correct this
    'ARS_maturity': [1,1000] #TODO: correct this
}
# load the samples from the original model
with open('observations.json') as json_file:
    observations = json.load(json_file)

def scale(x,factor): 
    """
    Coorects the fuzzy controller's output 
    x: fuzzy output
    factor: scale factor
    """
    if x >= 0.5:
        f=2*(factor-1)*(x-0.5)+1
        return f
    else:
        f = (1/factor)*(2*(factor-1)*x+1)
        return f
class Fuzzy_IL10:
    def __init__(self,params,above_48h):
        self.antecedents = []
        self.consequents = []
        self.params = params
        #// define antecedents
        # the marks of IL10 memberships
        IL10_intervals_below_48h= [0,0.01,10,self.params['IL10_det'],100]
        IL10_intervals_above_48h= [0,0.01,0.1,10,100]
        if above_48h:
            IL10_intervals = IL10_intervals_above_48h
        else:
            IL10_intervals = IL10_intervals_below_48h
        IL10 = ctrl.Antecedent(np.arange(IL10_intervals[0], IL10_intervals[-1], .05), 'IL10')
        print(IL10_intervals)
        IL10['Neg'] = fuzz.trimf(IL10.universe, [IL10_intervals[0], IL10_intervals[0],IL10_intervals[1]])
        IL10['Low'] = fuzz.trimf(IL10.universe, [IL10_intervals[0], IL10_intervals[1], IL10_intervals[2]])
        IL10['Stim'] = fuzz.trimf(IL10.universe, [IL10_intervals[1], IL10_intervals[2], IL10_intervals[3]])
        IL10['Det'] = fuzz.trapmf(IL10.universe, [IL10_intervals[2], IL10_intervals[3], IL10_intervals[-1], IL10_intervals[-1]])
        #// store
        self.antecedents.append(IL10)
#         IL10.view()
        #// define consequents
        range_value = np.arange(0, 1, .01)
        diff = ctrl.Consequent(range_value, 'diff')
        
        #// define membership functions
        sigma = .05
        diff_intervals = [0,self.params['diff_L'],.5,self.params['diff_H'], 1]
        diff['Z']=fuzz.gaussmf(range_value, diff_intervals[0], sigma)
        diff['L']=fuzz.gaussmf(range_value, diff_intervals[1], sigma)
        diff['M']=fuzz.gaussmf(range_value, diff_intervals[2], sigma)
        diff['H']=fuzz.gaussmf(range_value, diff_intervals[3], sigma)
        diff['VH']=fuzz.gaussmf(range_value, diff_intervals[4], sigma)
#         early_diff.view()

        #// Store
        self.consequents = [diff]
        #// rules
        rules = [
            ctrl.Rule(IL10['Stim'] , diff['VH']),
            ctrl.Rule(IL10['Low'] , diff['H']),
            ctrl.Rule(IL10['Neg'] , diff['M']),
            ctrl.Rule(IL10['Det'] , diff['L'])
        ]
        self.controler = ctrl.ControlSystemSimulation(ctrl.ControlSystem(rules))
        
    def forward(self,inputs):
        self.controler.input['IL10'] = inputs['IL10']
        self.controler.compute()
        # for item in self.consequents:
        #     item.view(sim=self.controler)
        outputs = self.controler.output
#         print(outputs)
        outputs['early_diff'] = outputs['diff'] 
        outputs['late_diff'] = outputs['diff'] 
        return outputs
def interactions(fs):
    return fs['f_IL10']

        
class Osteogenesis:
    def __init__(self,params):
        self.params = params
    def calculate_maturity(self,f_values,checkpoint):
        k0 = 1/(self.params['diff_time']*24) # base diff rate 1/hour
        f_early = f_values['early_diff'] 
        f_late = f_values['late_diff'] 
        maturity_t = self.params['maturity_t']*self.params['diff_time']*24 # because given maturity_t in the parameters is between 0 and 1 1
        k_early = scale(f_early,self.params['a_early_diff'])*k0
        k_late = scale(f_late,self.params['a_late_diff'])*k0
        
        if checkpoint < maturity_t:
            maturity = k_early*checkpoint # no for loop
            return maturity
        else:
            maturity_0 = k_early*maturity_t
            maturity_1 = k_late*(checkpoint-maturity_t)
            return maturity_0+maturity_1
               

    def simulate(self,inputs,measurement_scheme,exposure_time): 
        """
        simulates one single run 
        """
        if exposure_time > 48: # fuzzy controller is defined differently based on the exposure time
            above_48h = True
        else:
            above_48h = False
            
        self.fuzzy_IL10 = Fuzzy_IL10(self.params,above_48h=above_48h)
        f_IL10 = self.fuzzy_IL10.forward(inputs)
        f_values = {'f_IL10':f_IL10}
        final_f_values = interactions(f_values)
        
        ALP_checkpoint = measurement_scheme['ALP']*24 #time in hours, TODO: this is only one checkpoint right now
        ARS_checkpoint = measurement_scheme['ARS']*24
        maturity_ALP =  self.calculate_maturity(f_values=final_f_values,
                                            checkpoint = ALP_checkpoint)
        maturity_ARS =  self.calculate_maturity(f_values=final_f_values,
                                            checkpoint = ARS_checkpoint)
        ALP_sim = maturity_ALP*self.params['ALP_maturity']
        ARS_sim = maturity_ARS*self.params['ARS_maturity']
        return {'ALP':ALP_sim, 'ARS':ARS_sim} 
        return results
    def episode(self):
        """
        Returns results for the whole episode
        """
        results = {}
        for study in observations['studies']:
            results[study] = {}
        for study in observations['studies']:
            measurement_scheme = observations[study]['measurement_scheme']
            exposure_time = observations[study]['exposure_period']
            IDs = observations[study]['IDs']
            for ID in IDs:
                inputs = observations[study][ID]['inputs']
#                 print('ID: {}, inputs: {}'.format(ID,inputs))
                ID_results = self.simulate(inputs,measurement_scheme,exposure_time=exposure_time)
                results[study][ID] = ID_results
                
        return results
    def costs(self):
        results = self.episode()
        errors = {}
        for study,study_results in results.items():
            errors[study] = {}
            for ID, ID_results in study_results.items():
                ID_observations = observations[study][ID]['expectations']
                tag_errors = []
                for tag,tag_observation in ID_observations.items():
                    abs_diff =abs(ID_results[tag]-tag_observation['mean'])
                    means = [tag_observation['mean'],ID_results[tag]]
                    mean = np.mean(means)
                    tag_error = abs_diff/mean
                    tag_errors.append(tag_error)
                errors[study][ID] = np.mean(tag_errors)
        return errors
    def run(self):
        errors = self.costs()
        error = np.mean([np.mean(list(study_errors.values())) for study_errors in errors.values()])
        return error
with open('inferred_params.json') as file:
    inferred_params = json.load(file)

obj = Osteogenesis(params = inferred_params)

obj.episode()
# error = obj.run()


[0, 0.01, 10, 93.5576036565812, 100]
[0, 0.01, 10, 93.5576036565812, 100]
[0, 0.01, 10, 93.5576036565812, 100]
[0, 0.01, 10, 93.5576036565812, 100]


{'Valles_2020_part1': {'ctr': {'ALP': 203.80106977185858,
   'ARS': 415.4298624858345},
  'IL10_.1': {'ALP': 322.37481434045026, 'ARS': 657.1316084860036},
  'IL10_1': {'ALP': 328.67270632322476, 'ARS': 669.9692859490488},
  'IL10_10': {'ALP': 417.9539485293675, 'ARS': 851.9609418995409}}}

x 0 -> scaling factor 0.25
x .5 -> scaling factor: 1.0
x .75 -> scaling factor 2.5
x 1 -> scaling factor 4.0
